In [2]:
import pandas as pd
import numpy as np

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder

import re

In [3]:
url_data=pd.read_csv('train_url_only.csv')
print(url_data.shape)
url_data.tail()

(197202, 4)


,guid,description,typology,url
197197,0313dde2-b5bd-4a1a-b7ef-63436ad975ad,Фото ч/б матов. групповое с фигурными краями ...,фотографии и негативы,http://goskatalog.ru/muzfo-imaginator/rest/ima...
197198,669ecd91-4197-4734-a9a9-7cb79d4af9d3,Открытое письмо «Петродворец. Вид на Аллею фо...,предметы печатной продукции,http://goskatalog.ru/muzfo-imaginator/rest/ima...
197199,07ca9b7c-efea-49e3-8db8-e69e9a079bb4,Фото черно-белое. Художник Алексей Як. Кольцов...,фотографии и негативы,http://goskatalog.ru/muzfo-imaginator/rest/ima...
197200,c0df1cb4-1ceb-47ea-873e-6d96fc5a1a95,Грампластинка.,прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...
197201,139530c4-6d48-44ac-9ef7-da16dcacb3e1,Этикетка спичечная коллекционная. Амурский тигр,прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...


In [5]:
clean_description_big=pd.read_csv('clean_description_big.csv')
print(clean_description_big.shape)
clean_description_big.tail()

(179802, 2)


,description,enpology
179797,фото матов групповое фигурными краями подл ган...,14
179798,открытое письмо петродворец вид аллею фонтанов,8
179799,фото черно белое художник алексей кольцов фото...,14
179800,грампластинка,11
179801,этикетка спичечная коллекционная амурский тигр,11


In [6]:
clean_description_big.isnull().sum()

description    1128
enpology          0
dtype: int64

In [7]:
url_data = url_data.dropna()
url_data.isnull().sum()
url_data.shape

(179845, 4)

In [8]:
url_data = url_data[url_data.typology != 'предметы прикладного искусства, быта и этнографии ']
url_data = url_data[url_data.typology != 'предметы этнографии']
url_data = url_data[url_data.typology != 'Фотографии']
url_data = url_data[url_data.typology != 'документы, редкие книги']
url_data = url_data[url_data.typology != 'изделия прикладного искусства']

In [9]:
url_data['typology'].value_counts()

фотографии и негативы                                28766
документы                                            24178
предметы археологии                                  22569
предметы нумизматики                                 21570
предметы прикладного искусства, быта и этнографии    21326
предметы печатной продукции                          19739
графика                                              14064
прочие                                                8265
редкие книги                                          7681
предметы естественнонаучной коллекции                 4120
живопись                                              3400
предметы техники                                      1509
предметы минералогической коллекции                   1402
оружие                                                 709
скульптура                                             504
Name: typology, dtype: int64

In [10]:
le = LabelEncoder()
url_data['typology_encoded'] = le.fit_transform(url_data['typology'])
url_data.head()

,guid,description,typology,url,typology_encoded
1,43854438-31d8-4551-b3bb-73d44c02dc95,"Открытка почтовая: ""Воен. Гр. Дор. Гора Казбек"".",прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...,11
9,4ed27265-a5b0-4a60-8501-ed1652e2c9fc,"Портрет Павла Давыдовича Эттингера, гравюра",графика,http://goskatalog.ru/muzfo-imaginator/rest/ima...,0
11,aecd8cda-2fc3-4fe4-8031-6f5745b148cf,Открытка почтовая: Железноводск. Гора Железная.,прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...,11
14,20491552-ab61-4dd9-a979-b19254b78205,"Домик в Галерной гавани, гравюра",графика,http://goskatalog.ru/muzfo-imaginator/rest/ima...,0
19,49fa0182-c27e-4c22-9995-420ca1ba427e,Фотография членов с\х артели имени Подтелкова ...,прочие,http://goskatalog.ru/muzfo-imaginator/rest/ima...,11


In [12]:
dict(zip(url_data['typology'], url_data['typology_encoded']))

{'графика': 0,
 'документы': 1,
 'живопись': 2,
 'оружие': 3,
 'предметы археологии': 4,
 'предметы естественнонаучной коллекции': 5,
 'предметы минералогической коллекции': 6,
 'предметы нумизматики': 7,
 'предметы печатной продукции': 8,
 'предметы прикладного искусства, быта и этнографии': 9,
 'предметы техники': 10,
 'прочие': 11,
 'редкие книги': 12,
 'скульптура': 13,
 'фотографии и негативы': 14}

In [ ]:
url_data['typology_encoded'].value_counts()

14    28766
1     24178
4     22569
7     21570
9     21326
8     19739
0     14064
11     8265
12     7681
5      4120
2      3400
10     1509
6      1402
3       709
13      504
Name: typology_encoded, dtype: int64

In [ ]:
url_data = url_data.drop(['guid', 'typology', 'url'], axis=1)

In [ ]:
print(url_data.shape)
url_data.head(10)

(179802, 2)


,description,typology_encoded
1,"Открытка почтовая: ""Воен. Гр. Дор. Гора Казбек"".",11
9,"Портрет Павла Давыдовича Эттингера, гравюра",0
11,Открытка почтовая: Железноводск. Гора Железная.,11
14,"Домик в Галерной гавани, гравюра",0
19,Фотография членов с\х артели имени Подтелкова ...,11
29,Книга «Святцы»,12
31,"Казанский собор, гравюра",0
34,Приложение к диплому № 146652,1
36,Открытка почтовая: Железноводск. Пушкинская г...,11
49,Монета (номинал 1 рубль),7


In [ ]:
#Removing the html strips
def strip_html(text):
    return re.sub(r'http\S+', " ", text)

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_quotes(text):
    return re.sub(r'".*?"', '" "', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_between_quotes(text)
    return text
#Apply function on review column
url_data['description']=url_data['description'].apply(denoise_text)

In [ ]:
url_data['description'].head(10)

1                               Открытка почтовая: " ".
9           Портрет Павла Давыдовича Эттингера, гравюра
11      Открытка почтовая: Железноводск. Гора Железная.
14                     Домик в Галерной гавани, гравюра
19    Фотография членов с\х артели имени Подтелкова ...
29                                       Книга «Святцы»
31                             Казанский собор, гравюра
34                        Приложение к диплому № 146652
36    Открытка почтовая:  Железноводск. Пушкинская г...
49                             Монета (номинал 1 рубль)
Name: description, dtype: object

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)#знаки препинания
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'<*?>',' ', text)
    text = re.sub(r'br',' ', text) #любые символы
    text = re.sub(r'№',' ', text)
    text = re.sub(r'\n',' ', text)
    text = re.sub(r'  ', ' ', text)
    return text

url_data['description']=url_data['description'].apply(clean)

In [ ]:
url_data['description'].head(10)

1                                  открытка почтовая   
9            портрет павла давыдовича эттингера гравюра
11        открытка почтовая железноводск гора железная 
14                      домик в галерной гавани гравюра
19    фотография членов с х артели имени подтелкова ...
29                                        книга святцы 
31                              казанский собор гравюра
34                               приложение к диплому  
36    открытка почтовая  железноводск пушкинская гал...
49                               монета номинал  рубль 
Name: description, dtype: object

In [ ]:
url_data['description'] = url_data['description'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>2]))

In [ ]:
url_data.head(10)

,description,typology_encoded
1,открытка почтовая,11
9,портрет павла давыдовича эттингера гравюра,0
11,открытка почтовая железноводск гора железная,11
14,домик галерной гавани гравюра,0
19,фотография членов артели имени подтелкова крив...,11
29,книга святцы,12
31,казанский собор гравюра,0
34,приложение диплому,1
36,открытка почтовая железноводск пушкинская галл...,11
49,монета номинал рубль,7


In [ ]:
text = url_data['description'].to_list()

In [ ]:
text[:10]
print(len(text))

179802


In [ ]:
!pip install -U spacy -qq

     |████████████████████████████████| 6.4 MB 3.7 MB/s 
     |████████████████████████████████| 621 kB 50.9 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 456 kB 35.9 MB/s 
     |████████████████████████████████| 10.1 MB 37.6 MB/s 


In [ ]:
import spacy
from spacy.lang.ru.examples import sentences

In [ ]:
spacy.__version__

'3.1.1'

In [ ]:
!python -m spacy download ru_core_news_lg -qq

2021-08-09 08:45:31.090105: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 514.2 MB 19 kB/s 
     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 8.2 MB 5.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
nlp = spacy.load("ru_core_news_lg")

In [ ]:
lemmatized_text = []
for desc in text:
    lemmatized_text.append(' '.join([word.lemma_ for word in nlp(desc)]))

KeyboardInterrupt: ignored

In [ ]:
lemmatized_text[:10]

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from string import punctuation
import unicodedata
import collections

In [ ]:
nltk.download('stopwords')
stopword_list_r=nltk.corpus.stopwords.words('russian')
stopword_list_e=nltk.corpus.stopwords.words('english')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
swr = stopwords.words("russian")
swe = stopwords.words('english')
# reg_tokenizer = RegexpTokenizer(pattern=r"\s+")
reg_tokenizer = RegexpTokenizer(pattern=r"\w+")

In [ ]:
NoStopwords = [] 
for i, element in enumerate(text):
    if i % 2000 == 0: print(i, end = ' ')
    NoStopwords.append(' '.join([word for word in element.split() if word not in swr and word not in swe]))
print(len(NoStopwords))
NoStopwords[:5]

0 2000 4000 6000 8000 10000 12000 14000 16000 18000 20000 22000 24000 26000 28000 30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 50000 52000 54000 56000 58000 60000 62000 64000 66000 68000 70000 72000 74000 76000 78000 80000 82000 84000 86000 88000 90000 92000 94000 96000 98000 100000 102000 104000 106000 108000 110000 112000 114000 116000 118000 120000 122000 124000 126000 128000 130000 132000 134000 136000 138000 140000 142000 144000 146000 148000 150000 152000 154000 156000 158000 160000 162000 164000 166000 168000 170000 172000 174000 176000 178000 179802


['открытка почтовая',
 'портрет павла давыдовича эттингера гравюра',
 'открытка почтовая железноводск гора железная',
 'домик галерной гавани гравюра',
 'фотография членов артели имени подтелкова кривошлыкова год']

In [ ]:
clean_description_big_1 = pd.concat([pd.Series(NoStopwords, name = 'description'), pd.Series(url_data['typology_encoded'].to_list(), name = 'enpology')], axis=1)
# clean_description_big.to_csv('clean_description_big.csv', index=False)

In [ ]:
# tokenized_text = reg_tokenizer.tokenize_sents(lemmatized_text)
# tokenized_text[:5]

In [ ]:
# NoStopwords = [] 
# for i, element in enumerate(tokenized_text):
#     if i % 2000 == 0: print(i, end = ' ')
#     NoStopwords.append(' '.join([word for word in element if word not in swr and word not in swe]))
# NoStopwords[:5]

In [ ]:
# clean_description_token = pd.concat([pd.Series(NoStopwords, name = 'description'), pd.Series(train_500_data['enpology'], name = 'enpology')], axis=1)

In [ ]:
# clean_description_token.to_csv('clean_description_token', index=False)

In [ ]:
print(clean_description_big_1.head(10))

                                         description  enpology
0                                  открытка почтовая        11
1         портрет павла давыдовича эттингера гравюра         0
2       открытка почтовая железноводск гора железная        11
3                      домик галерной гавани гравюра         0
4  фотография членов артели имени подтелкова крив...        11
5                                       книга святцы        12
6                            казанский собор гравюра         0
7                                 приложение диплому         1
8  открытка почтовая железноводск пушкинская галл...        11
9                               монета номинал рубль         7


In [ ]:
clean_description_big=pd.read_csv('clean_description_big.csv')
print(clean_description_big.shape)
clean_description_big.tail()

(179802, 2)


,description,enpology
179797,фото матов групповое фигурными краями подл ган...,14
179798,открытое письмо петродворец вид аллею фонтанов,8
179799,фото черно белое художник алексей кольцов фото...,14
179800,грампластинка,11
179801,этикетка спичечная коллекционная амурский тигр,11


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 3.8 MB/s 


In [ ]:
!pip install transformers -qq

     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 636 kB 39.4 MB/s 
     |████████████████████████████████| 3.3 MB 18.3 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape) #312 мерный 

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(312,)


In [ ]:
features = clean_description_big_1['description'].apply((lambda x: embed_bert_cls(x, model, tokenizer)))

In [ ]:
len(features[0])

312

In [ ]:
df = pd.DataFrame(features.to_list(), columns=range(312))
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311
0,-0.021714,-0.042040,0.058279,-0.007260,-0.022157,0.038823,0.075788,0.003425,0.007822,-0.045026,0.097639,-0.022283,-0.000058,0.056528,0.020711,0.005967,-0.012928,0.033693,0.014006,0.072415,-0.035411,-0.001385,0.035000,-0.041480,0.113270,0.029279,-0.000751,0.018261,-0.026207,0.080020,-0.039796,-0.083942,0.070224,0.031431,-0.039355,0.087231,0.009140,0.004344,-0.095304,0.040902,...,-0.003273,0.037928,0.038069,0.040785,-0.002436,0.012110,0.003205,-0.051811,0.071631,0.030810,-0.104069,0.070484,0.017481,-0.023256,-0.069520,-0.038369,0.019155,-0.028036,0.014075,-0.008126,-0.016815,0.019641,-0.065312,0.051364,-0.131673,0.020942,-0.027850,-0.060535,0.040308,-0.056175,-0.005532,0.087189,-0.016329,-0.001204,0.049405,-0.016313,-0.030909,0.090894,0.085891,-0.072096
1,-0.004629,-0.042766,0.067375,-0.096717,0.015186,0.031365,-0.000049,0.057014,-0.071349,-0.007636,0.060555,0.023652,0.018690,0.101489,0.016760,0.017042,0.087517,-0.001846,0.029794,0.030333,-0.081542,0.018864,-0.023720,-0.025173,0.052445,0.032674,-0.007992,0.030628,0.017732,0.069653,-0.025783,0.022933,0.020840,0.013799,-0.037835,0.026814,0.009025,-0.056899,-0.117882,0.000839,...,-0.029580,0.045931,0.011230,-0.050689,0.034871,0.051835,0.018020,-0.009878,0.163682,0.039266,-0.034396,-0.000188,0.008033,0.005968,0.008929,0.003683,-0.029168,0.022518,0.075145,-0.009141,0.011022,-0.026685,-0.068805,-0.001855,-0.018709,0.072498,-0.042119,-0.067993,-0.021241,-0.003962,0.016582,0.124419,0.027207,0.015517,-0.009407,-0.000959,-0.051667,0.052932,0.007319,-0.114169
2,-0.019716,-0.038397,0.041032,-0.098718,0.025730,0.060552,0.024531,-0.004538,-0.030425,-0.072059,0.119869,-0.014058,0.005465,0.037740,0.034146,0.030692,0.004778,0.050407,0.004992,0.046895,-0.023857,-0.012468,0.018083,-0.071955,0.134229,0.025433,0.028505,0.039533,0.038130,0.092615,-0.029866,-0.022946,0.041717,0.052782,-0.024938,0.092983,-0.012938,0.005142,-0.102123,0.011810,...,-0.009816,0.017616,-0.000131,0.017999,0.036571,0.020096,0.009167,-0.006803,0.115777,-0.004109,-0.090463,0.019014,0.012626,-0.001332,-0.021302,-0.060607,0.019530,0.031620,0.000694,0.010891,-0.064190,0.010779,-0.095627,0.023837,-0.084793,0.020181,-0.012581,-0.069850,0.024224,-0.011057,0.001191,0.112578,-0.007709,0.020618,0.065250,-0.007783,-0.023703,0.040241,0.107656,-0.071257
3,-0.024952,-0.010374,0.068869,-0.070954,0.022150,0.053929,0.052644,0.046649,-0.035743,-0.036959,0.046963,-0.002650,0.005017,0.063040,0.016682,0.022022,0.087568,0.041525,0.022144,0.041212,-0.073730,-0.013436,-0.024336,-0.066921,0.069440,0.104658,0.004891,0.023838,-0.009325,0.065358,-0.057531,-0.020577,0.039386,0.053488,-0.008994,0.096150,0.016551,-0.034235,-0.126810,0.021680,...,-0.022575,0.010238,0.031019,-0.044451,0.062841,0.009866,0.022896,-0.012370,0.098666,0.024355,-0.044603,0.040729,-0.029721,-0.046783,-0.046932,-0.015393,-0.026052,-0.005283,0.037689,-0.005070,0.007285,-0.008600,-0.096406,0.013079,-0.086202,0.053493,-0.054767,-0.083388,-0.015044,-0.013344,0.021869,0.139338,0.027304,0.006924,0.006272,-0.012233,-0.030984,0.094089,0.047679,-0.078607
4,0.019844,-0.070693,-0.000085,-0.062304,-0.009906,0.045926,-0.021093,-0.010436,-0.027714,-0.018598,0.039704,-0.032597,-0.001419,0.056551,0.011976,0.036398,0.007728,-0.036512,0.015921,0.092876,-0.047502,0.012011,-0.027772,-0.050664,0.120199,0.040934,0.053072,0.042103,0.052571,0.085796,-0.071029,0.045712,0.045968,0.003975,-0.079383,0.064510,0.051110,-0.040950,-0.030486,-0.011293,...,-0.032107,0.039201,0.070408,-0.003537,-0.005644,0.029254,-0.025635,-0.032903,0.091567,0.056763,-0.089783,0.006580,0.000206,0.016613,0.023188,0.042304,-0.024841,0.050274,0.067967,0.009988,0.063644,-0.061733,-0.014438,0.017616,-0.025389,0.031376,-0.094808,-0.075471,-0.032125,0.024324,0.019302,0.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [ ]:
clean_description_big=pd.read_csv('clean_description_big.csv')
print(clean_description_big.shape)
clean_description_big.tail()

(179802, 2)


,description,enpology
179797,фото матов групповое фигурными краями подл ган...,14
179798,открытое письмо петродворец вид аллею фонтанов,8
179799,фото черно белое художник алексей кольцов фото...,14
179800,грампластинка,11
179801,этикетка спичечная коллекционная амурский тигр,11


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(df,  clean_description_big['enpology'])

In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf_pred = lr_clf.predict(test_features)

In [ ]:
len(lr_clf_pred), len(test_labels)

(44951, 44951)

In [ ]:
lr_f1_score = f1_score(test_labels.values, lr_clf_pred, average='weighted')

In [ ]:
lr_f1_score

0.6300774414141161

In [ ]:
import xgboost as xgb

In [ ]:
clf_xgb = xgb.XGBClassifier(max_depth=10, n_estimators=100, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)
clf_xgb.fit(train_features, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.07, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
import pickle

In [ ]:
with open('model_for_text_xgboost.pkl','wb') as f:
    pickle.dump(clf_xgb,f)

In [ ]:
# load
with open('model_for_text_xgboost.pkl', 'rb') as f:
    clf_xgb = pickle.load(f)

In [ ]:
xgb_pred = clf_xgb.predict(test_features)

In [ ]:
xgb_pred

array([ 1,  7,  1, ..., 14,  8,  4])

In [ ]:
type(xgb_pred)

numpy.ndarray

In [ ]:
type(test_labels.values)

numpy.ndarray

In [ ]:
f1 = f1_score(test_labels.values, xgb_pred, average='weighted')
f1

0.8933173131540203

In [ ]:
test_data=pd.read_csv('test_desc_only.csv')

In [ ]:
print(test_data.shape)
test_data.tail()

(676, 4)


,Unnamed: 0,guid,description,typology
671,1215,b6e5611f-ce3f-4cd6-99ca-a6e8e529c2bb,Монета. 15 копеек,NaN
672,1216,7f84a349-fa7c-4dee-bb47-9974d0b30888,Монета. 10 копеек,NaN
673,1217,bf0e560c-535f-479e-97c0-9b3d7c577b3a,Монета. 10 копеек,NaN
674,1220,2519dcdf-3cb2-44cc-9a39-80ff704ca56d,Монета. 10 копеек,NaN
675,1222,3884278d-de78-4510-a077-d0223fef8ab2,Монета. 15 копеек,NaN
